## MERGE

In [1]:
import numpy as np
import pandas as pd
import os

pd.set_option('display.max_columns', None)

path = os.getcwd()

In [2]:
df_census = pd.read_csv(os.path.join(path, 'csv/census_by_city.csv'))     # (25235, 35)
df_crime = pd.read_csv(os.path.join(path, 'csv/fbi_crime_uscities.csv'))  # (8103, 16)
df_rent = pd.read_csv(os.path.join(path, 'csv/rent_cleaned.csv'))         # (2831, 4)
df_poll = pd.read_csv(os.path.join(path, 'csv/pollution_summary.csv'))    # (845, 20)
df_lat_lon = pd.read_excel(os.path.join(path, 'csv/uscities.xlsx'))       # (28338, 17)

In [3]:
# replace 'State_Id' in df_census so it matches the other columns
df_census.rename(columns = {'State_Id':'State'}, inplace = True)
df_lat_lon.rename(columns = {'state_id':'State','city': 'City'}, inplace = True)
df_lat_lon = df_lat_lon[['City', 'State', 'lat', 'lng']]

In [4]:
# put all the dfs in a list
df_lst = [df_census, df_crime, df_rent, df_poll, df_lat_lon]

# remove whitespace in all the dfs
for x in df_lst:
    x['City'] = x['City'].str.strip()
    x['State'] = x['State'].str.strip()

# make sure it worked    
for x in df_lst:
    print(x['City'][0])
    print(x['State'][0])

Aaronsburg
PA
Anchorage
AK
Aleutians East
AK
Anchorage
AK
Adak
AK


In [6]:
# merging the data and following its shape sequentially
df_net = pd.merge(df_census, df_crime, on=['City', 'State'])
print(df_net.shape)
df_net2 = pd.merge(df_net, df_rent, on=['City', 'State'])
print(df_net2.shape)
df_net3 = pd.merge(df_net2, df_poll, on=['City', 'State'])
print(df_net3.shape)
df_net4 = pd.merge(df_net3, df_lat_lon, on=['City', 'State'])
print(df_net4.shape)

(6867, 51)
(769, 53)
(432, 71)
(426, 73)


## CLEANUP

### Update Crime Ratings 
- comparison is across fewer cities. 
- run pd.qcut again

In [7]:
df_net4['Crime Rating'].value_counts()

High      334
Medium     84
Low         8
Name: Crime Rating, dtype: int64

crime rating skews very high because after the merging there were only a little over 400 cities versus the 8000 cities reported in the FBI crime data. 

In [8]:
# dropping old crime rating column
df_net4 = df_net4.drop(columns = ['Crime Rating'])

In [9]:
# Create a new crime rating by using qcut() on the number of merged cities we have
df_net4['Crime Rating'] = pd.qcut(df_net4['Crime Rate (per 1000 residents)'], q=3, labels=['Low', 'Medium', 'High'])

In [10]:
df_net4['Crime Rating'].value_counts()

High      142
Medium    142
Low       142
Name: Crime Rating, dtype: int64

### Clean Columns

In [11]:
col = ["Poverty", "ChildPoverty", "Unemployment", "Professional", "Office", "Service", "Construction", "Production"]
df_net4[col] = df_net4[col].round(2)

### Reorder Columns

In [12]:
df_net4.columns

Index(['TractId', 'City', 'State', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'Unnamed: 0_x',
       'Population', 'Violent crime', 'Murder and nonnegligent manslaughter',
       'Rape', 'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny- theft', 'Motor vehicle theft', 'Arson',
       'Crime Rate (per 1000 residents)', 'Unnamed: 0_y', 'Rent', 'Year',
       'Days with AQI', 'Good Days', 'Moderate Days',
       'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
       'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th 

In [13]:
df_net4 = df_net4[[ 
    'City', 'State', 
    'lat', 'lng',
    'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
    'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr','IncomePerCap', 
    'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Employed', 'Unemployment', 
    'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Professional', 
    'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 
    'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 
    'Rent', 'Year',
    'Population','Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
    'Robbery', 'Aggravated assault', 'Property crime', 'Burglary','Larceny- theft', 
    'Motor vehicle theft', 'Arson', 'Crime Rate (per 1000 residents)', 'Crime Rating',
    'Days with AQI', 'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 
    'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 
    '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2', 'Days Ozone', 'Days SO2', 
    'Days PM2.5', 'Days PM10', 'Level of Concern',
    ]]

In [14]:
df_net4.rename(columns = {'Level of Concern':'Air Quality Index',
                         'Crime Rate (per 1000 residents)': 'Crime Rate per 1000',
                          'lng': 'lon'}, inplace = True)

### Drop Duplicates

In [15]:
df_net4[df_net4.duplicated(subset=['City', 'State', 'lat', 'lon'], keep=False)]

,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index
24,Austin,TX,30.3004,-97.7522,1038282,524260,513917,32.5,51.0,6.8,0.2,6.9,0.0,14813195,2212708,8195920,1120821,14.4,18.2,579384,4.4,78.2,14.3,7.4,0.1,48.7,15.4,22.1,7.5,6.2,74.2,9.2,3.5,2.0,2.4,8.6,24.9,1549,2020,986062,3953,32,534,971,2416,36588,4344,29216,3028,103,41.11,Medium,183,135,47,0,1,0,0,153,59,44,1,3,85,0,94,0,Good
25,Austin,TX,30.3004,-97.7522,1038282,524260,513917,32.5,51.0,6.8,0.2,6.9,0.0,14813195,2212708,8195920,1120821,14.4,18.2,579384,4.4,78.2,14.3,7.4,0.1,48.7,15.4,22.1,7.5,6.2,74.2,9.2,3.5,2.0,2.4,8.6,24.9,1046,2020,986062,3953,32,534,971,2416,36588,4344,29216,3028,103,41.11,Medium,183,135,47,0,1,0,0,153,59,44,1,3,85,0,94,0,Good
34,Beloit,WI,42.5230,-89.0184,47855,23079,24771,16.1,67.4,10.9,0.2,1.3,0.0,605389,95152,297632,35064,21.9,33.4,20773,9.2,85.1,10.7,4.0,0.1,24.2,17.8,22.8,7.7,27.3,81.9,10.4,0.4,2.3,1.8,2.9,21.9,907,2020,37025,154,4,19,31,100,1114,122,921,71,11,34.25,Medium,99,90,6,3,0,0,0,105,50,40,0,0,99,0,0,0,Good
35,Beloit,WI,42.5230,-89.0184,47855,23079,24771,16.1,67.4,10.9,0.2,1.3,0.0,605389,95152,297632,35064,21.9,33.4,20773,9.2,85.1,10.7,4.0,0.1,24.2,17.8,22.8,7.7,27.3,81.9,10.4,0.4,2.3,1.8,2.9,21.9,907,2020,7707,7,0,2,2,3,126,23,95,8,0,17.26,Low,99,90,6,3,0,0,0,105,50,40,0,0,99,0,0,0,Good
51,Bristol,TN,36.5572,-82.2154,36467,17658,18805,1.7,93.2,2.6,0.0,0.6,0.0,421324,64560,230059,27101,16.7,26.5,15667,7.0,80.3,11.7,7.7,0.2,32.8,18.0,26.5,8.1,14.3,85.5,8.5,0.5,0.9,1.1,3.3,22.5,704,2020,26900,135,1,15,12,107,813,138,604,71,3,35.24,Medium,192,174,16,2,0,0,0,109,50,38,0,0,99,46,47,0,Good
52,Bristol,TN,36.5572,-82.2154,36467,17658,18805,1.7,93.2,2.6,0.0,0.6,0.0,421324,64560,230059,27101,16.7,26.5,15667,7.0,80.3,11.7,7.7,0.2,32.8,18.0,26.5,8.1,14.3,85.5,8.5,0.5,0.9,1.1,3.3,22.5,704,2020,26900,135,1,15,12,107,813,138,604,71,3,35.24,Medium,192,174,16,2,0,0,0,109,50,38,0,0,99,46,47,0,Good
53,Bristol,TN,36.5572,-82.2154,36467,17658,18805,1.7,93.2,2.6,0.0,0.6,0.0,421324,64560,230059,27101,16.7,26.5,15667,7.0,80.3,11.7,7.7,0.2,32.8,18.0,26.5,8.1,14.3,85.5,8.5,0.5,0.9,1.1,3.3,22.5,704,2020,26900,135,1,15,12,107,813,138,604,71,3,35.24,Medium,192,174,16,2,0,0,0,109,50,38,0,0,99,46,47,0,Good
54,Bristol,TN,36.5572,-82.2154,36467,17658,18805,1.7,93.2,2.6,0.0,0.6,0.0,421324,64560,230059,27101,16.7,26.5,15667,7.0,80.3,11.7,7.7,0.2,32.8,18.0,26.5,8.1,14.3,85.5,8.5,0.5,0.9,1.1,3.3,22.5,704,2020,26900,135,1,15,12,107,813,138,604,71,3,35.24,Medium,192,174,16,2,0,0,0,109,50,38,0,0,99,46,47,0,Good
88,Conway,AR,35.0754,-92.4694,82433,40485,41936,4.6,76.2,14.6,0.2,1.8,0.0,835231,143736,419890,72869,17.1,17.7,40133,4.8,79.3,15.7,4.8,0.1,37.6,17.2,25.3,9.2,10.4,83.2,9.1,0.1,2.4,1.5,3.5,21.9,898,2020,67336,324,2,49,51,222,1842,177,1567,98,8,32.17,Medium,275,207,66,2,0,0,0,132,59,42,1,4,138,0,132,0,Good
89,Conway,AR,35.0754,-92.4694,82433,40485,41936,4.6,76.2,14.6,0.2,1.8,0.0,835231,143736,419890,72869,17.1,17.7,40133,4.8,79.3,15.7,4.8,0.1,37.6,17.2,25.3,9.2,10.4,83.2,9.1,0.1,2.4,1.5,3.5,21.9,703,2020,67336,324,2,49,51,222,1842,177,1567,98,8,32.17,Medium,275,207,66,2,0,0,0,132,59,42,1,4,138,0,132,0,Good


In [16]:
df_net4 = df_net4.drop_duplicates(subset=['City', 'State', 'lat', 'lon'], keep='first')

### Save

In [17]:
print(df_net4.shape)
df_net4.head()

(397, 69)


,City,State,lat,lon,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate per 1000,Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index
0,Akron,OH,41.0798,-81.5219,267782,129623,138086,1.9,67.4,23.2,0.2,3.6,0.0,3068406,504125,1732548,258991,19.6,26.2,126247,8.5,85.4,10.3,4.1,0.1,32.3,19.8,25.3,6.6,15.6,83.3,8.5,3.1,1.6,0.6,2.7,21.9,911,2020,197882,1782,27,181,328,1246,6568,1686,4305,577,65,42.20,High,274,203,68,3,0,0,0,107,63,43,0,0,132,0,142,0,Good
1,Albany,GA,31.5776,-84.1762,93421,43571,49835,2.8,29.2,65.2,0.1,0.9,0.0,1025141,199233,547674,89500,29.6,41.9,35704,16.2,71.9,23.7,4.1,0.1,30.7,21.4,23.1,8.1,16.3,78.1,12.4,1.8,2.3,0.7,4.3,19.1,814,2020,74989,790,12,32,165,581,3452,729,2489,234,17,56.57,High,244,182,60,2,0,0,0,114,63,37,0,0,0,0,244,0,Good
2,Albany,NY,42.6664,-73.7987,147794,70664,77096,7.8,60.3,20.1,0.1,7.6,0.0,1917135,331452,1102265,158632,17.4,17.8,74293,6.6,71.6,24.3,3.9,0.0,42.9,20.5,25.2,4.6,6.6,70.4,7.2,10.1,7.3,1.7,2.9,18.9,1205,2020,97221,736,4,60,189,483,2919,445,2315,159,16,37.59,Medium,274,240,34,0,0,0,0,75,53,39,1,0,169,0,104,0,Good
3,Albany,OR,44.6274,-123.0966,61555,30378,31168,11.4,82.7,0.6,0.7,1.9,0.1,542299,75753,268177,35254,14.3,18.6,27721,8.2,78.5,15.9,5.4,0.1,33.3,18.4,22.7,9.9,15.5,79.0,10.7,0.3,2.6,2.0,5.1,20.8,1197,2020,54993,70,1,10,16,43,1467,128,1242,97,17,27.95,Medium,182,161,21,0,0,0,0,70,51,19,0,0,0,0,182,0,Good
4,Albuquerque,NM,35.1053,-106.6464,632240,309415,322758,50.4,38.5,2.5,4.0,2.4,0.1,7693803,1270581,4099669,581026,18.2,24.8,292907,6.8,74.5,19.6,5.6,0.2,39.8,20.0,24.0,8.3,7.7,80.2,9.1,1.8,1.8,2.5,4.3,21.7,1013,2020,561920,7596,84,486,1699,5327,26059,0,20634,5425,98,59.89,High,183,101,82,0,0,0,0,100,65,50,1,0,127,0,25,30,Good


In [18]:
# extract the merged df as a .csv
df_net4.to_csv('merged.csv', index = False) 